<a href="https://colab.research.google.com/github/tylee33/DLforFinance/blob/master/SectorCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**네이버 증권에서 펀드 섹터별 수익률 정보 크롤링**

In [1]:
import requests
import bs4
import pandas as pd
import numpy as np

def __init__():
    df=[]
    url=[]

def GetDf(url):
    r = requests.get(url)
    html = r.text

    soup = bs4.BeautifulSoup(html, 'html.parser')
    allRows = soup.find('tbody').find_all('tr')

    results = [[data.text.split('?')[0].replace('\n','').replace('\t','') for data in td.find_all('td')]\
               for td in allRows]

    rowspan = [] #row

    for tr_no, tr in enumerate(allRows): #enumerate returns index, data 
        tmp = []
        for td_no, data in enumerate(tr.find_all('td')):
            if data.has_attr("rowspan"):
                rowspan.append((tr_no, td_no, int(data["rowspan"]), data.get_text().split('?')[0].replace('\n','').replace('\t','')))

    if rowspan:
        for i in rowspan:
            for j in range(1, i[2]):
                results[i[0]+j].insert(i[1],i[3])

    headers=[]

    for i in range(1,11):
        if i==6:
            continue
        if i==9:
            headers.append(soup.find('thead').find('th',{'class':'th_result'+str(i)+'_on'}).text)
        else:
            headers.append(soup.find('thead').find('th',{'class':'th_result'+str(i)}).text)    
    
    df = pd.DataFrame(data=results)
    
    df.columns = headers
    
    df['설정액'] = df['설정액'].str.replace(',','').astype(float)
    df['1개월 증감액'] = df['1개월 증감액'].str.replace(',','').astype(float)
    df = df.replace('N/A',np.nan)
    df.dropna(inplace=True)
    
    df = df.apply(pd.to_numeric, args =('ignore',))
    
    df.reset_index(inplace=True, drop=True)
    return df

GetDf('http://finance.naver.com/fund/fundTypeEarningRate.nhn?ivstAreaWorldYn=N')

,대유형,소유형,펀드수,설정액,1개월 증감액,올해,1개월,3개월,1년
0,주식형,배당주식,55,28250.0,127.0,3.50,4.00,1.96,-13.91
1,주식형,K200인덱스,78,118581.0,5997.0,4.48,5.11,-0.08,-14.86
2,주식형,일반주식,322,114256.0,264.0,2.90,3.80,-1.32,-16.87
3,주식형,중소형주식,53,17932.0,-123.0,1.15,2.84,-3.67,-18.23
4,주식혼합형,일반주식혼합,92,35951.0,-107.0,2.70,3.45,0.70,-8.31
5,주식혼합형,공격적자산배분,32,6695.0,-62.0,3.05,4.63,-1.63,-11.03
6,채권혼합형,일반채권혼합,229,69375.0,834.0,0.70,1.05,0.29,-3.49
7,채권혼합형,보수적자산배분,38,3706.0,9.0,0.78,0.79,-0.18,-2.96
8,채권형,중기채권,61,32315.0,1127.0,-0.02,-0.14,1.61,4.95
9,채권형,우량채권,29,13637.0,1511.0,0.00,-0.08,1.19,3.70


In [2]:
GetDf('http://finance.naver.com/fund/fundTypeEarningRate.nhn?ivstAreaWorldYn=Y')

,대유형,소유형,펀드수,설정액,1개월 증감액,올해,1개월,3개월,1년
0,해외주식형,브라질주식,8,745.0,-36.0,9.92,12.50,12.23,6.11
1,해외주식형,기타국가주식,5,554.0,2.0,6.87,5.36,10.76,-7.55
2,해외주식형,인도주식,26,3153.0,96.0,-0.64,-2.01,7.35,-13.51
3,해외주식형,아시아신흥국주식,17,2863.0,18.0,3.24,2.85,5.01,-17.15
4,해외주식형,남미신흥국주식,5,939.0,-5.0,10.18,11.69,4.53,-6.38
5,해외주식형,아시아태평양주식(ex J),24,7192.0,94.0,4.67,4.08,4.10,-11.52
6,해외주식형,글로벌신흥국주식,47,8433.0,-45.0,6.00,5.48,3.42,-14.81
7,해외주식형,동남아주식,16,2177.0,1.0,5.64,4.22,2.39,-11.13
8,해외주식형,프론티어마켓주식,1,21.0,-0.0,2.52,1.44,2.15,-6.04
9,해외주식형,중국주식,160,56075.0,181.0,6.33,5.43,2.03,-25.09


**펀드 섹터별 수익률 계산**

In [3]:
def __init__(self):
    df
    
def GetCalc(df):
    ###########################################################
    # drop 조건
    # 1. 설정액이 1000억이하 섹터 drop.
    # 2. 한번이라도 수익률이 마이너스를 기록했던 섹터 drop.
    # 3. 국내채권,국내주식,해외채권,해외주식,부동산,현금으로 나누고 이 6개의 비중을 퍼지이론으로 조절하는 방식.
    ###########################################################
    
    df = df.drop(df[(df['설정액']<1000)].index) # 1번 조건 drop.
    
    df['MoMofQoQ'] = ((((100+df['3개월'])/100)**(1/3))-1)*100
    df['MoMofYoY'] = ((((100+df['1년'])/100)**(1/12))-1)*100    
    df = df.drop(df[(df.MoMofYoY<0)|(df.MoMofQoQ<0)|(df['1개월']<0)].index) # 2번 조건 drop.
    
    dfStd = df[['1개월','MoMofYoY','MoMofQoQ']].std(axis=1)
    df['Safety'] = (100-((dfStd - dfStd.min())/(dfStd.max()-dfStd.min()))*100)
    
    ReturnAdjValue = 1 # 수익성향보정치
    SafetyAdjValue = 1 # 안전성향보정치
    
    # 기대수익률 = (1개월, MoMofYoY, MoMofQoQ의 평균 수익률 * 수익성향보정치) * (안전도*안전성향보정치)
    df['기대수익률'] = (((df[['1개월','MoMofYoY','MoMofQoQ']].mean(axis=1)*ReturnAdjValue) * (df['Safety']*SafetyAdjValue) / 10000 + 1)**12-1)*100
    df = df.drop(['3개월','1년'],axis=1)
    
    df = df.sort_values(by=['기대수익률'], ascending=False)
    #df.reset_index(inplace=True, drop=True)
    return df

GetCalc(GetDf('http://finance.naver.com/fund/fundTypeEarningRate.nhn?ivstAreaWorldYn=N'))

,대유형,소유형,펀드수,설정액,1개월 증감액,올해,1개월,MoMofQoQ,MoMofYoY,Safety,기대수익률
14,부동산형,부동산임대,7,3392.0,0.0,0.34,0.54,0.428164,0.558363,87.734174,5.490658
10,채권형,일반채권,85,65125.0,3227.0,0.13,0.16,0.239426,0.222263,93.328777,2.345707
11,채권형,초단기채권,29,72513.0,299.0,0.14,0.20,0.179677,0.153694,96.953933,2.088221
16,절대수익추구형,공모주하이일드,6,2742.0,-83.0,0.21,0.56,0.089919,0.247438,54.804543,1.985004
12,MMF,MMF,134,877123.0,119836.0,0.09,0.15,0.143128,0.134829,100.000000,1.725324
15,절대수익추구형,채권알파,58,7835.0,185.0,0.20,0.49,0.119856,0.098630,58.545884,1.671833
13,부동산형,부동산대출채권,11,7050.0,0.0,0.43,0.58,1.620272,1.112654,0.000000,0.000000


**계산된 수익률을 바탕으로 퍼지이론 적용**

In [4]:
def RuleCalc(propensity):
    ########################################
    # Rule evaluation - The rules are different according to user's propensity
    # propensity 1 - conservative propensity
    # rule1
    # IF safety / low
    # and returns / high
    # then importance / low
    #
    # rule2
    # IF safety / high
    # and returns / low
    # then importance / high
    #
    # rule3
    # IF safety / middle
    # and returns / high
    # then importance / middle
    #########################################
    if propensity=='conservative':
        setHigh=[]
        for index, x in fuzzyDf.iterrows():
            setHigh.append(x['SafetyHigh']*x['returnLow'])
        fuzzyDf['setHigh']=setHigh

        setMid=[]
        for index, x in fuzzyDf.iterrows():
            setMid.append(x['SafetyMid']*x['returnHigh'])
        fuzzyDf['setMid']=setMid

        setLow=[]
        for index, x in fuzzyDf.iterrows():
            setLow.append(x['SafetyLow']*x['returnHigh'])
        fuzzyDf['setLow']=setLow
        return fuzzyDf['setHigh'],fuzzyDf['setMid'],fuzzyDf['setLow']
    ########################################
    # propensity 2 - moderate propensity
    # rule1
    # IF safety / low
    # and returns / high
    # then importance / middle
    #
    # rule2
    # IF safety / high
    # and returns / low
    # then importance / low
    #
    # rule3
    # IF safety / middle
    # and returns / high
    # then importance / high
    #########################################
    elif propensity=='moderate':
        setHigh=[]
        for index, x in fuzzyDf.iterrows():
            setHigh.append(x['SafetyMid']*x['returnHigh'])
        fuzzyDf['setHigh']=setHigh

        setMid=[]
        for index, x in fuzzyDf.iterrows():
            setMid.append(x['SafetyHigh']*x['returnLow'])
        fuzzyDf['setMid']=setMid

        setLow=[]
        for index, x in fuzzyDf.iterrows():
            setLow.append(x['SafetyLow']*x['returnHigh'])
        fuzzyDf['setLow']=setLow
        return fuzzyDf['setHigh'],fuzzyDf['setMid'],fuzzyDf['setLow']
    
    ########################################
    # propensity 3 - aggresive propensity
    # rule1
    # IF safety / low
    # and returns / high
    # then importance / high
    #
    # rule2
    # IF safety / high
    # and returns / low
    # then importance / low
    #
    # rule3
    # IF safety / middle
    # and returns / high
    # then importance / middle
    #########################################
    elif propensity=='aggressive':
        setHigh=[]
        for index, x in fuzzyDf.iterrows():
            setHigh.append(x['SafetyLow']*x['returnHigh'])
        fuzzyDf['setHigh']=setHigh

        setMid=[]
        for index, x in fuzzyDf.iterrows():
            setMid.append(x['SafetyMid']*x['returnHigh'])
        fuzzyDf['setMid']=setMid

        setLow=[]
        for index, x in fuzzyDf.iterrows():
            setLow.append(x['SafetyHigh']*x['returnLow'])
        fuzzyDf['setLow']=setLow
        return fuzzyDf['setHigh'],fuzzyDf['setMid'],fuzzyDf['setLow']

domesticDf=GetDf('http://finance.naver.com/fund/fundTypeEarningRate.nhn?ivstAreaWorldYn=N')
overseasDf=GetDf('http://finance.naver.com/fund/fundTypeEarningRate.nhn?ivstAreaWorldYn=Y')

earningRateDf = domesticDf.append(overseasDf, ignore_index=True)
earningRateDf = GetCalc(earningRateDf)

domesticDf=GetCalc(domesticDf)
overseasDf=GetCalc(overseasDf)

domesticStockDf = domesticDf[domesticDf['대유형'].str.contains("주식")].reset_index(drop=True)
domesticBondDf= domesticDf[domesticDf['대유형'].str.contains("채권")].reset_index(drop=True)

overseaStockDf = overseasDf[overseasDf['대유형'].str.contains("주식")].reset_index(drop=True)
overseaBondDf = overseasDf[overseasDf['대유형'].str.contains("채권")].reset_index(drop=True)

realestateDf =  earningRateDf[earningRateDf['대유형'].str.contains("부동산")].reset_index(drop=True)
cashDf =  earningRateDf[earningRateDf['대유형'].str.contains("MMF")].reset_index(drop=True)

resultDf = pd.DataFrame(columns=earningRateDf.columns)
dfList = [domesticStockDf,domesticBondDf,overseaStockDf,overseaBondDf,realestateDf,cashDf]

for x in dfList:
    if x.empty==False:
        resultDf = resultDf.append(x.iloc[0])

fuzzyDf=pd.DataFrame()
fuzzyDf['대유형']=resultDf['대유형']

###############################################
# Fuzzyfication
# x를 계산하는 함수는 설문조사 혹은 인공지능에 따라 함수 변경 가능
# x/15-(16/3)같은 부분을 함수화 시켜 차후 DB와 연동.
##############################################
safetyHigh=[]
for x in resultDf.Safety:
    if x>80 and x<95:
        safetyHigh.append(x/15-(16/3))
    elif x>=95:
        safetyHigh.append(1)
    else:
        safetyHigh.append(0)
fuzzyDf['SafetyHigh']=safetyHigh

safetyMid=[]
for x in resultDf.Safety:
    if x>50 and x<=75:
        safetyMid.append(x/25-2) 
    elif x>75:
        safetyMid.append(-(x/25)+4)
    else:
        safetyMid.append(0)
fuzzyDf['SafetyMid']=safetyMid

safetyLow=[]
for x in resultDf.Safety:
    if x>=20 and x<=(160/3):
        safetyLow.append(-(x*3/100)+(8/5)) 
    elif x<20:
        safetyLow.append(1)
    else:
        safetyLow.append(0)
fuzzyDf['SafetyLow']=safetyLow

returnHigh=[]
for x in resultDf['기대수익률']:
    if x>=2 and x<8:
        returnHigh.append(x/6-(1/3)) 
    elif x>=8:
        returnHigh.append(1)
    else:
        returnHigh.append(0)
fuzzyDf['returnHigh']=returnHigh

returnLow=[]
for x in resultDf['기대수익률']:
    if x>=2 and x<4:
        returnLow.append(-(x/4)+1) 
    elif x<2:
        returnLow.append(1)
    else:
        returnLow.append(0)
fuzzyDf['returnLow']=returnLow

propensity = 'conservative'
fuzzyDf['setHigh'],fuzzyDf['setMid'],fuzzyDf['setLow'] = RuleCalc(propensity)

##############################################
# Defuzzification uses COG
##############################################
fuzzyDf['COG']=\
fuzzyDf['setLow']*(10+20)+fuzzyDf[['setMid','setLow']].max(axis=1)*(30+40)\
+fuzzyDf['setMid']*50+fuzzyDf[['setHigh','setMid']].max(axis=1)*(60+70)+fuzzyDf['setHigh']*(80+90+100)

resultDf['percentage']=(fuzzyDf['COG']/fuzzyDf['COG'].sum()).round(2)
resultReturn = resultDf['기대수익률']*resultDf['percentage']
print (pd.DataFrame(resultDf[['소유형','기대수익률','percentage']]))

print (propensity, 'expected return:',resultReturn.sum()) # expected return

     소유형     기대수익률  percentage
0   일반채권  2.345707        0.24
0  부동산임대  5.490658        0.12
0    MMF  1.725324        0.64
conservative expected return: 2.3260560559819057
